# QPQ Redshift Measurements

In [27]:
# imports
from xastropy.xguis import spec_guis as xsg
from xastropy.sdss import quasars as sdssq
from xastropy.sdss import qso as sdssqso

import multiprocessing

from linetools.spectra import io as lsio
import qpq_z as qpqz

## Following Hewitt & Wild 2010

Their prescription, applied to SDSS spectra, is as follows
(in order of preference):

**Redshift recipe:**

1. Gassian fits to [OIII]
1. Gaussian fit to [OII] -- Effective wavelength = 3728.60A
1. Cross-correlation with MgII
  * Minimum rest-wavelength of 1975A for $z<1.1$
  * Lower minimum for $z>1.1$ which includes CIII]
  * Residuals are at the $\Delta z \, / (1+z) \approx 10^{-5}$ level
1. Cross-correlation dominated by CIII]
  * Required for $z>2$ SDSS spectra
  * Must correct for luminosity bias assessed from $1.1<z<2$ quasars
1. Cross-correlation including CIV
  * Used when CIII] is too weak
  * Strong luminosity bias with broken slopes is applied
  * One must also include a correction for CIV absorption 
  
**Systematics:**

1. [OIII] is offset by 45 km/s bluewards from CaII in $z<0.4$ AGN
1. [OII] is offset by 24 km/s redwards of [OIII]
1. Alternative template for FIRST detected, especially for CIII]

**Cross-correlation:**

1. Avoid strong emission-lines near edge of spectrum
1. Define an identical window for quasar and template spectrum
1. Narrow absorption features are flagged from a continuum-subtracted spectrum
    * Continuum is defined with a 61-pixel median filter
    * Those falling 4.5$\sigma$ below are flagged with grow radius of 2 pixels
1. Restrict rest-frame wavelength range:
    * Pixels with $\lambda_r > 7000$A are excluded
    * Pixels near the brightest sky lines are masked
    * Pixels with $\lambda_r < 1275$A are excluded
    * They have a series of other rules, mainly for SDSS
1. Calculate the cross-correlation lag:
$cc(l) = \frac{ \Sigma_i Q_i T_i/\sigma_i^2}{\sqrt{\Sigma_i (Q_i/\sigma_i)^2 \Sigma_i (T_i/\sigma_i)^2}}$

    * $Q$ is the continuum-subtracted quasar spectrum. They use a 601 pixel, median filter
    * $T$ is the template, also continuum-subtracted
    * $\sigma_i$ is the noise array
2. Fit a quadratic to $cc(l)$ over $\pm 100$ pixels
3. Iteratively reduce the fit until it is over only $\pm 20$ pixels
4. Strive for $cc_{max} > 0.2$

---
## Modifying for QPQ

### Key concerns

1. Should we convert their SDSS pixel intervals to km/s intervals?
    * Yes
1. How sensitive will our results be to our fluxing?
    * Continuum subtraction should help
    * Consider using our own
1. How should we handle bright sky lines?
    * Mask only the ones that H&W do
1. How should we mask in the near-IR?
    * N/A.  Not worth it for the small sample with MgII alone
1. Should we attempt to stitch optical and near-IR spectra together?
    * N/A
1. How shall we rebin the templates to our non-SDSS data?
    * Rebin to data wavelength grid
1. How shall we rebin our data to a constant velocity grid?
    * Avoid

## Exploring the template

In [2]:
#Read
hw10 = QTable.read('hw10_template.dat',format='ascii',names=['wave','flux','N','FD','NFD'])
hw10[0:5]

wave,flux,N,FD,NFD
float64,float64,int64,float64,int64
1275.26,8.131,179,-999.0,0
1275.56,7.582,186,-999.0,0
1275.85,7.845,191,-999.0,0
1276.15,7.78,196,-999.0,0
1276.44,7.673,204,-999.0,0


In [3]:
# Plot
reload(xsg)
chk=False
if chk:
    xsg.run_xspec((hw10['wave'],hw10['flux']))

## Test Spectrum (DR7)

In [162]:
reload(lsio)
reload(qpqz)
spec = lsio.readspec('/u/xavier/SDSS/DR7_QSO/spectro/1d_26/0650/1d/spSpec-52143-0650-199.fit')
#z_dict = qpqz.init_zdict(zguess=1.845)
#z_dict = qpqz.init_zdict(zguess=1.8519)#1.8507)
HW = qpqz.HewittWildz(spec,1.845)
HW

{'zinit': 1.845, 'wvmax': <Quantity 0.0 Angstrom>, 'wvmin': <Quantity 0.0 Angstrom>, 'zguess': 1.845}

## Wavelength Window

In [163]:
reload(qpqz)
HW.set_window()
HW

{'zinit': 1.845, 'wvmax': <Quantity 3241.2888053576944 Angstrom>, 'wvmin': <Quantity 1364.1 Angstrom>, 'zguess': 1.845}

## Mask

In [164]:
reload(qpqz)
HW.set_mask()
HW.mask

array([False, False, False, ..., False, False, False], dtype=bool)

## Apply mask

In [160]:
#cut_spec = qpqz.apply_mask(spec, mask)
HW.apply_mask()

AttributeError: 'HewittWildz' object has no attribute 'mask'

In [161]:
chk=False
if chk:
    HW.cut_spec.plot()

## Rebin template

In [149]:
#new_templ = qpqz.rebin_template(cut_spec.dispersion/(1+z_dict['zguess']))
HW.rebin_template()

In [150]:
#Check
chk=False
if chk:
    xdb.xplot(hw10['wave'],hw10['flux'], xtwo=HW.new_templ.dispersion, ytwo=HW.new_templ.flux)
# Looks good

## Continuum subtract

In [151]:
HW.sub_spec = HW.continuum_subtract(HW.cut_spec)
HW.sub_templ = HW.continuum_subtract(HW.new_templ)
#
chk=False
if chk:
    xdb.xplot(HW.sub_spec.dispersion, HW.sub_spec.flux, HW.sub_templ.flux)

## Cross-correlate

In [153]:
HW.cross_correlate()
#HW.cc

## Fit

In [80]:
reload(qpqz)
HW.fit_cc(cc)

## Loop

In [81]:
reload(qpqz)
nlags = [100,80,60,40,20]
olag = 0.
for nlag in nlags:
    iolag = int(np.round(olag))
    # Cross-correlate
    cc = qpqz.cross_correlate(sub_spec, z_dict, sub_templ,olag=iolag,nlag=nlag)
    # Centre
    centre = qpqz.fit_cc(cc)#,debug=True)
    # Rest olag
    olag += centre
    print('olag={:g}'.format(olag))

olag=6.66591
olag=7.58112
olag=7.48768
olag=5.20349
olag=0.849906


## Final answer

In [82]:
dv_SDSS = 69.021818 * u.km/u.s
newz = z_dict['zguess'] + (1+z_dict['zguess'])*(dv_SDSS*olag)/const.c.to('km/s')
newz

<Quantity 1.852458047339095>

---
## Simple Class Test

In [179]:
reload(qpqz)
spec = lsio.readspec('/u/xavier/SDSS/DR7_QSO/spectro/1d_26/0650/1d/spSpec-52143-0650-199.fit')
HW = qpqz.HewittWildz(spec,1.845)
HW

{'zinit': 1.845, 'wvmax': <Quantity 0.0 Angstrom>, 'wvmin': <Quantity 0.0 Angstrom>, 'zguess': 1.845}

In [180]:
HW.run()
HW

{'wvmax': <Quantity 3234.8053563977755 Angstrom>, 'zguess': 1.8507021706899576, 'wvmin': <Quantity 1364.1 Angstrom>, 'zinit': 1.845, u'zfin': 1.851909860303605, u'cc_max': 0.35159941560007851}

---
## DR7 Test

### HW10 Table 4
Taken from Vizier.  Appears updated

In [11]:
#HW10 Table 4
fil = os.getenv('QPQ9')+'Analysis/Redshifts/hw10_tab4.fits.gz'
hw10_tab4 = QTable.read(fil)
hw10_tab4[0:3]

SDSS,z,e_z,Det,z1,n_z,Plate,MJD,Fiber,Sloan,DR7,RAJ2000,DEJ2000
,,,,,,,d,,,,deg,deg
string152,float32,float32,int16,float32,uint8,int16,float64,int16,string40,string24,float64,float64
J000006.53+003055.2,1.8232,0.001,-1,nan,3,685,52203.0,467,Sloan,DR7,0.02723,0.51534
J000008.13+001634.6,1.8363,0.0006,-1,nan,3,685,52203.0,470,Sloan,DR7,0.0339,0.2763
J000009.26+151754.5,1.1974,0.0004,0,1.2,2,751,52251.0,354,Sloan,DR7,0.0386,15.29848


In [8]:
# Parse
zmnx = [1.5,2.2]
gdq = np.where( (hw10_tab4['z']>zmnx[0]) & (hw10_tab4['z']<zmnx[1]))[0]
print('We have {:d} DR7 quasars to try.'.format(len(gdq)))

We have 30370 DR7 quasars to try.


### Load up those in my SDSS DR7 dataset

In [17]:
reload(sdssq)
sdss_dr7 = sdssq.SdssQuasars(verbose=False)

In [23]:
def load_qso(ii):
    pf = (hw10_tab4['Plate'][ii],hw10_tab4['Fiber'][ii])
    return sdss_dr7[pf[0],pf[1]]

In [24]:
#tst_qsos = [sdss_dr7[hw10_tab4['Plate'][igd],hw10_tab4['Fiber'][igd]] for igd in gdq[0:100]]
tst_qsos = map(load_qso,gdq[0:10])

In [36]:
# Multi-processing
pool = multiprocessing.Pool(4) # initialize thread pool N threads
tst_qsos = pool.map(load_qso,gdq[0:20])

In [35]:
len(tst_qsos)

30370

In [10]:
print(multiprocessing.cpu_count())

8


### Sandbox

In [6]:
reload(qpqz)
new_templ = qpqz.rebin_template(spec.dispersion/(1+z_dict['zguess']))

In [30]:
dwv = spec.dispersion - np.roll(spec.dispersion,1)
dv = const.c.to('km/s')*dwv/spec.dispersion
dv

<Quantity [ -4.27854475e+05,  6.90218177e+01,  6.90218177e+01,...,
             6.90218177e+01,  6.90218177e+01,  6.90218177e+01] km / s>

In [5]:
spec.plot()

In [26]:
nlag = 100
lags = -1*nlag + np.arange(2*nlag+1)

In [27]:
lags

array([-100,  -99,  -98,  -97,  -96,  -95,  -94,  -93,  -92,  -91,  -90,
        -89,  -88,  -87,  -86,  -85,  -84,  -83,  -82,  -81,  -80,  -79,
        -78,  -77,  -76,  -75,  -74,  -73,  -72,  -71,  -70,  -69,  -68,
        -67,  -66,  -65,  -64,  -63,  -62,  -61,  -60,  -59,  -58,  -57,
        -56,  -55,  -54,  -53,  -52,  -51,  -50,  -49,  -48,  -47,  -46,
        -45,  -44,  -43,  -42,  -41,  -40,  -39,  -38,  -37,  -36,  -35,
        -34,  -33,  -32,  -31,  -30,  -29,  -28,  -27,  -26,  -25,  -24,
        -23,  -22,  -21,  -20,  -19,  -18,  -17,  -16,  -15,  -14,  -13,
        -12,  -11,  -10,   -9,   -8,   -7,   -6,   -5,   -4,   -3,   -2,
         -1,    0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
         10,   11,   12,   13,   14,   15,   16,   17,   18,   19,   20,
         21,   22,   23,   24,   25,   26,   27,   28,   29,   30,   31,
         32,   33,   34,   35,   36,   37,   38,   39,   40,   41,   42,
         43,   44,   45,   46,   47,   48,   49,   